In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
pd.options.mode.chained_assignment = None 
import os

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

'\nThis code was adapted from \n'

In [1]:
# -*- coding: utf-8 -*-
"""
Implementation of the pre-processing code used by Smith et al., 2015, Nature
Neuroscience (available at https://www.fmrib.ox.ac.uk/datasets/HCP-CCA/).
Assumes you have access to both the unrestricted AND restricted behavioral data
from HCP. And can load it into a dataframe + re-order it according to
instructions at the above link (see `smith_cols.csv` for up-to-date column
names that work with the S1200 release of HCP.
License: Apache 2.0 (https://www.apache.org/licenses/LICENSE-2.0)
"""

import numpy as np
import pandas as pd
from scipy import special as spc


def palm_inormal(X, c=None, method='blom', quanti=False):
    """
    Parameters
    ----------
    X : array_like
    c : float, optional
    method : {'blom', 'tukey', 'bliss', 'waerden', 'solar'}, optional
    quanti : bool, optional
    Returns
    -------
    Z : numpy.ndarray
    """

    methods = dict(
        blom=(3 / 8),
        tukey=(1 / 3),
        bliss=(1 / 2),
        waerden=0,
        solar=0
    )
    if method not in methods:
        raise ValueError('Provided method {} invalid. Must be one of {}.'
                         .format(method, methods))
    if c is None:
        c = methods.get(method, 3 / 8)

    if quanti:
        iX = np.argsort(X)
        ri = np.argsort(iX)
        N = len(X)
        p = ((ri - c) / (N - 2 * c + 1))
        Z = np.sqrt(2) * spc.erfinv(2 * p - 1)
    else:
        Z = np.ones_like(X) * np.nan

        for x in range(X.shape[-1]):
            XX = X[:, x]
            ynan = ~np.isnan(XX)
            XX = XX[ynan]

            iX = np.argsort(XX)
            ri = np.argsort(iX)

            N = len(XX)
            p = ((ri + 1 - c) / (N - 2 * c + 1))
            Y = np.sqrt(2) * spc.erfinv(2 * p - 1)

            U, IC = np.unique(Y, return_inverse=True)
            if U.size < N:
                sIC = np.sort(IC)
                dIC = np.diff(np.vstack((sIC, 1)))
                U = np.unique(sIC[dIC == 0])
                for u in range(len(U)):
                    Y[IC == U[u]] = np.mean(Y[IC == U[u]])

            Z[ynan, x] = Y

    return Z


def nets_normalise(X, axis=None):
    """
    Parameters
    ----------
    X : (N, M) array_like
    axis : int, optional
    Returns
    -------
    Xnorm : (N, M) numpy.ndarray
    """

    X = np.asarray(X)
    if axis is None:
        if len(X) > 1:
            axis = 0
        elif X.ndim > 1 and len(X) == 1:
            axis = 1

    X = (X - np.nanmean(X, axis=axis)) / np.nanstd(X, ddof=1, axis=axis)
    X[np.isinf(X)] = 0

    return X


def nets_demean(X, axis=None):
    """
    Parameters
    ----------
    X : (N, M) array_like
    axis : int, optional
    Returns
    -------
    Xdm : (N, M) numpy.ndarray
    """

    X = np.asarray(X)
    if axis is None:
        if len(X) > 1:
            axis = 0
        elif X.ndim > 1 and len(X) == 1:
            axis = 1

    X = X - np.nanmean(X, axis=axis)

    return X


def to_indicator(X):
    """
    Converts `X` to an indicator variable matrix
    Parameters
    ----------
    X : (N,) array_like
    Returns
    -------
    I : (N, P) numpy.ndarray
    """

    X = np.asarray(X)
    if X.ndim > 1:
        raise ValueError('Cannot convert multi-dimensional array to indicator')

    return np.column_stack([X == u for u in np.unique(X)])


def nearestPD(A):
    """
    Find the nearest positive-definite matrix to input
    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1]_, which
    credits [2]_. Taken from [3]_
    Parameters
    ----------
    A : (N, N) array_like
        Input square matrix to be converted to nearest symmetric positive
        definite matrix
    References
    ----------
    .. [1] www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd
    .. [2] N.J. Higham. (1998). Computing a nearest symmetric positive
       semidefinite matrix. doi.org/10.1016/0024-3795(88)90223-6
    .. [3] gist.github.com/fasiha/fdb5cec2054e6f1c6ae35476045a0bbd
    """

    def isPD(B):
        """Returns true when input is positive-definite, via Cholesky"""
        try:
            _ = np.linalg.cholesky(B)
            return True
        except np.linalg.LinAlgError:
            return False

    B = (A + A.T) / 2
    _, s, V = np.linalg.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(np.linalg.norm(A))
    identity = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(np.linalg.eigvals(A3)))
        A3 += identity * (-mineig * k**2 + spacing)
        k += 1

    return A3


def get_badvars(behavior):
    """
    Parameters
    ----------
    behavior : (N, M) array_like
    Returns
    -------
    badvars : numpy.ndarray
        Indices of variables that should be dropped from `behavior`
    """

    behavior = np.asarray(behavior)

    badvars = []
    for i in range(behavior.shape[-1]):
        Y = behavior[:, i]
        grotKEEP = np.logical_not(np.isnan(Y))
        Ygrot = Y[grotKEEP]
        if len(Ygrot) == 0 or np.std(Ygrot, ddof=1) == 0:
            badvars.append(i)
            continue

        grot = (Ygrot - np.median(Ygrot)) ** 2
        grot = np.max(grot / np.mean(grot))
        Y_indic = to_indicator(Ygrot)

        good = (np.sum(grotKEEP) > 250
                and np.std(Ygrot, ddof=1) > 0
                and (grot < 100)
                and np.max(np.sum(Y_indic, axis=0)) / len(Ygrot) < 0.95)
        if not good:
            badvars.append(i)

    return np.asarray(badvars)


def smith_prep(behavior, Nkeep=100, return_pcs=True):
    """
    Pre-processed `behavior` according to guidelines from [1]_
    Parameters
    ----------
    behavior : (N, 478) pandas.DataFrame
        Input behavioral data from HCP for `N` participants. The 478 columns
        should match those specified in `smith_cols.csv`, which is expected to
        be in the same directory as this script
    Nkeep : int, optional
        Number of PCs to retain if `return_pcs` is True. Default: 100
    return_pcs : bool, optional
        Whether to return principal components of behavior or original
        behavioral variables. Default: True
    Returns
    -------
    df : (N, M) pandas.DataFrame
        Where `M` is `Nkeep` if `return_pcs` is True; otherwise, it will depend
        on the number of "good" behavioral variables
    conf : (N, P) pandas.DataFrame
        Confounder variables. Regress out of brain / behavior, if you want.
        If `return_pcs=True`, they have already been regressed out of behavior
    References
    ----------
    .. [1] Smith, S. M., Nichols, T. E., Vidaurre, D., Winkler, A. M., Behrens,
       T. E., Glasser, M. F., ... & Miller, K. L. (2015). A positive-negative
       mode of population covariation links brain connectivity, demographics
       and behavior. Nature neuroscience, 18(11), 1565.
    """

    cols = np.loadtxt('smith_cols.csv', dtype=str)
    if np.all(behavior.columns != cols):
        raise ValueError('Provided `behavior` dataframe must have exactly '
                         '478 columns in the order specified by Smith et al., '
                         '2015.')

    subj_ids = np.asarray(behavior['Subject'])
    cols = np.asarray(behavior.columns)

    behavior = np.asarray(behavior, dtype=float)
    conf = palm_inormal(np.column_stack([
        behavior[:, [13, 14, 21, 22, 24]],
        behavior[:, [264, 265]] ** (1 / 3)
    ]))
    conf[np.isnan(conf)] = 0
    conf = nets_normalise(np.column_stack([conf, conf ** 2]))

    varskeep = np.setdiff1d(
        range(behavior.shape[-1]),
        np.hstack([
            0, 5, range(265, 457), 1, 6, 13, 14, 21, 22, 24, 264, 265,
            10, 11, 12, 16, 18, 26, 28, 30, 33, 39, 203, 204, range(211, 223),
            range(228, 233), 235, 237, 241, 476, 2, 3, 4, 7, 8, 9, 15, 17, 19,
            20, 23, 25, 27, 29, 31, 32, range(34, 39), 457, 458, 459, 462, 463,
            get_badvars(behavior)
        ])
    )

    varsd = palm_inormal(behavior[:, varskeep])
    for i in range(varsd.shape[-1]):
        grot = np.isnan(varsd[:, i]) == 0
        grotconf = nets_demean(conf[grot])
        tmp = grotconf @ (np.linalg.pinv(grotconf) @ varsd[grot, i])
        var = varsd[grot, i] - tmp
        varsd[grot, i] = var / np.linalg.norm(var)

    # calculate covariance matrix subject-subject covariance matrix
    mvarsd = np.ma.masked_array(varsd, mask=np.isnan(varsd))
    varsdCOV = np.asarray(np.ma.cov(mvarsd))

    varsdCOV2 = nearestPD(varsdCOV)
    uu = varsd
    df = pd.DataFrame(varsd, columns=cols[varskeep], index=subj_ids)

    if return_pcs:
        dd, uu = np.linalg.eig(varsdCOV2)
        dd, uu = np.real(dd), np.real(uu)
        i = np.argsort(dd)[::-1]
        uu = uu[:, i][:, :Nkeep]

        uu2 = uu - conf @ (np.linalg.pinv(conf) @ uu)

        df = pd.DataFrame(uu2, index=subj_ids,
                          columns=[f'PC{n}' for n in range(1, Nkeep + 1)])

    return df, pd.DataFrame(conf, index=subj_ids)

In [4]:
def abcd_dataloader(file):
    '''
    This generic function loads data from the ABCD study and
    ------
    Input:
    - ABCD file name without .txt entension as string, eg, "abcd_ksad01" 
    ------
    Output:
    - DATAFRAMEs for baseline and follow up data
    '''
    # load header info and store in dictionary
    header = pd.read_csv("../ABCDTabular/{}.txt".format(file),
                         header=None, sep='\t', nrows=2)  # read header
    # load data and use header info to name columns
    data = pd.read_csv("../ABCDTabular/{}.txt".format(file),
                       header=None, sep='\t', skiprows=2)  # read data
    data.columns = list(header.iloc[0, :])

    # drop duplicates
    df = data.drop_duplicates(
        subset=["subjectkey", "interview_date", "interview_age"], ignore_index=False)
    # select subjects with baseline, 1y follow up, 2y follow up data
    df_base = df.loc[df["eventname"] == 'baseline_year_1_arm_1']
    df_follow = df.loc[df["eventname"] == '2_year_follow_up_y_arm_1']
    
    return df_base, df_follow

In [5]:
def female_categorical(row, care_or_youth):
    '''
    female: body hair growth + breast development || using menarche info as follows:
    prepubertal = 2 + no menarche
    early pubertal = 3 + no menarche
    midpubertal =>3 + no menarche
    late pubertal <=7 + menarche
    postpubertal = 8 + menarche
    according to Herting et al (2021) Frontiers in Endocrinology
    '''
    ### based on caregiver or youth report, menarche variable is labeled differently
    if care_or_youth == "youth":
        menarche = row["pds_f5_y"]
    elif care_or_youth == "caregiver":
        menarche = row["pds_f5b_p"]
    
    if np.isnan(row["PDS_cat_score"])==False:    
        if menarche == 1.0:
            if row["PDS_cat_score"] == 2.0:
                return "prepubertal"
            if row["PDS_cat_score"] == 3.0:
                return "early pubertal"
            if row["PDS_cat_score"] >= 3.0:
                return "midpubertal"
            
        elif menarche == 4.0:
            if row["PDS_cat_score"] <= 7.0:
                return "late pubertal"
            if row["PDS_cat_score"] == 8.0:
                return "postpubertal"
                     
    elif np.isnan(row["PDS_cat_score"])==True:
        return np.nan
    

def male_categorical(row):
    '''
    male: body hair growth + facial hair + voice change
    prepubertal = 3 x
    early pubertal = 4 or 5 (no 3 point response)x
    midpubertal = 6-8 (no point 4 response) x
    late pubertal = 9-11 
    postpubertal = 12 (all 4 point)
    according to Herting et al (2021) Frontiers in Endocrinology
    with minor adjustment to not create cases for which category is not
    well defined (see paper)
    '''
    
    if np.isnan(row["PDS_cat_score"])==False:
        if row["PDS_cat_score"] == 3.0:
            return "prepubertal"
        
        if 4.0 <= row["PDS_cat_score"] <= 5.0:
            return "early pubertal"

        if 6.0 <= row["PDS_cat_score"] <= 8.0:
            return "midpubertal"
   
        if 9.0 <= row["PDS_cat_score"] <= 11.0:
                return "late pubertal"
            
        if row["PDS_cat_score"] == 12.0:
            return "postpubertal"
        
    elif np.isnan(row["PDS_cat_score"])==True:
        return np.nan

In [6]:
def calc_pubertal_scores(timepoint, care_or_youth):
    
    ''''
    This function calculates 'summary' scores derived from the PDS Scale:
    - gonadal vs adrenal processes
    - mean PDS
    - PDS categorical score + pubertal stage 
    -----
    Input:
    - file: ABCD file containing information about perceived puberty without .txt extension. Default: caregiver report
    - timepoint: (str) from ['1_year_follow_up_y_arm_1', 'baseline_year_1_arm_1','2_year_follow_up_y_arm_1']
    -----
    Output:
    - dataframe: file + 'summary' pubertal scores
    '''
        
    # who replied? 
    if care_or_youth == "caregiver":
        file = "abcd_ppdms01"
    elif care_or_youth =="youth":
        file = "abcd_ypdms01"
        
    
    ## load file 
    path = "../ABCDTabular/{}.txt"
    f = path.format(file)
    
    header = pd.read_csv(f, header=None, sep='\t', nrows=2)
    df = pd.read_csv(f, header=None, sep='\t', skiprows=2)
    df.columns = list(header.iloc[0, :])
    
    df = df.drop_duplicates(
        subset=["subjectkey", "interview_date", "interview_age"], ignore_index=False)
    
    df_t = df.loc[df["eventname"]==timepoint] ## choose timepoint
    df_t.replace(999.0, np.nan, inplace=True) ## replace all "dont know" with np.nan
    df_t.replace(777.0, np.nan, inplace=True) ## replace all "refuse to answer" with np.nan -- only for youth
    
    
    fem = df_t.loc[df_t.sex=="F"] ## subset for sex
    men = df_t.loc[df_t.sex=="M"]
    
    ## calculate puberty scores based on youth report
    if care_or_youth == "youth":
    
        ## calculate gonadal puberty scores 
        fem["gonadal"]= fem[["pds_ht2_y", "pds_f4_2_y", "pds_f5_y"]].mean(axis=1, skipna=False)
        men["gonadal"] = men[["pds_ht2_y","pds_m4_y", "pds_m5_y"]].mean(axis=1, skipna=False)

        ## calculate adrenal puberty scores
        fem["adrenal"]= fem[["pds_skin2_y", "pds_bdyhair_y"]].mean(axis=1, skipna=False)
        men["adrenal"]= men[["pds_skin2_y", "pds_bdyhair_y"]].mean(axis=1, skipna=False)

        ## calculate average PDS scores
        fem["PDS_mean"] =  fem[["pds_ht2_y", "pds_f4_2_y", "pds_f5_y", 
                                "pds_skin2_y", "pds_bdyhair_y"]].mean(axis=1, skipna=False)
        men["PDS_mean"] =  men[["pds_ht2_y","pds_m4_y", "pds_m5_y",
                                "pds_skin2_y", "pds_bdyhair_y"]].mean(axis=1, skipna=False)

        ## calculate PDS category score
        fem["PDS_cat_score"] = fem[["pds_f4_2_y", "pds_bdyhair_y"]].sum(axis=1, skipna=False)
        men["PDS_cat_score"] = men[["pds_bdyhair_y", "pds_m5_y", "pds_m4_y"]].sum(axis=1, skipna=False)

        ## transform PDS category scores to pubertal stages
        fem["PDS_category"] = fem.apply(lambda row: female_categorical(row, care_or_youth="youth"), axis=1)
        men["PDS_category"] = men.apply(lambda row: male_categorical(row), axis=1)
        
    
    ## calculate puberty scores based on caregiver report
    elif care_or_youth =="caregiver":
        
        ## calculate gonadal puberty scores 
        fem["gonadal"]= fem[["pds_1_p", "pds_f4_p", "pds_f5b_p"]].mean(axis=1, skipna=False)
        men["gonadal"] = men[["pds_1_p","pds_m4_p", "pds_m5_p"]].mean(axis=1, skipna=False)

        ## calculate adrenal puberty scores
        fem["adrenal"]= fem[["pds_3_p", "pds_2_p"]].mean(axis=1, skipna=False)
        men["adrenal"]= men[["pds_3_p", "pds_2_p"]].mean(axis=1, skipna=False)

        ## calculate average PDS scores
        fem["PDS_mean"] =  fem[["pds_1_p", "pds_f4_p", "pds_f5b_p", "pds_3_p", "pds_2_p"]].mean(axis=1, skipna=False)
        men["PDS_mean"] =  men[["pds_1_p","pds_m4_p", "pds_m5_p", "pds_3_p", "pds_2_p"]].mean(axis=1, skipna=False)

        ## calculate PDS category score
        fem["PDS_cat_score"] = fem[["pds_f4_p", "pds_2_p"]].sum(axis=1, skipna=False)
        men["PDS_cat_score"] = men[["pds_2_p", "pds_m5_p", "pds_m4_p"]].sum(axis=1, skipna=False)

        ## transform PDS category scores to pubertal stages
        fem["PDS_category"] = fem.apply(lambda row: female_categorical(row, care_or_youth="caregiver"), axis=1)
        men["PDS_category"] = men.apply(lambda row: male_categorical(row), axis=1)
        
    ## combine both datasets and return 
    return pd.concat([fem, men])

In [42]:
puberty_y_df = calc_pubertal_scores(timepoint= '2_year_follow_up_y_arm_1', care_or_youth="youth")

In [43]:
puberty_y_df

,collection_id,abcd_ypdms01_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,sex,eventname,pds_sex_y,...,pds_remote___4,mentrualcycle11_y,pds_device,menstrualcycle10_y,collection_title,gonadal,adrenal,PDS_mean,PDS_cat_score,PDS_category
2,2573,72383,47118,NDAR_INV042UJKFB,NDAR_INV042UJKFB,12/10/2018,150,F,2_year_follow_up_y_arm_1,NaN,...,0,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,3.333333,3.5,3.4,6.0,late pubertal
16,2573,72397,47118,NDAR_INV04GAB2AA,NDAR_INV04GAB2AA,08/10/2020,155,F,2_year_follow_up_y_arm_1,NaN,...,1,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,NaN,3.0,NaN,6.0,None
19,2573,72400,47118,NDAR_INV04JY6730,NDAR_INV04JY6730,10/11/2019,136,F,2_year_follow_up_y_arm_1,NaN,...,0,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,1.666667,2.5,2.0,7.0,midpubertal
23,2573,72404,47118,NDAR_INV04P0G6LK,NDAR_INV04P0G6LK,10/25/2020,156,F,2_year_follow_up_y_arm_1,NaN,...,1,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,3.333333,3.5,3.4,6.0,late pubertal
29,2573,72410,47118,NDAR_INV04TRXUGL,NDAR_INV04TRXUGL,12/18/2019,143,F,2_year_follow_up_y_arm_1,NaN,...,0,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,1.666667,2.0,1.8,4.0,midpubertal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39743,2573,112124,47118,NDAR_INVZW8ENNPU,NDAR_INVZW8ENNPU,03/01/2020,143,M,2_year_follow_up_y_arm_1,NaN,...,0,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,1.000000,1.5,1.2,3.0,prepubertal
39746,2573,112127,47118,NDAR_INVZW8G4W5A,NDAR_INVZW8G4W5A,06/22/2019,149,M,2_year_follow_up_y_arm_1,NaN,...,0,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,2.666667,2.5,2.6,8.0,midpubertal
39752,2573,112133,47118,NDAR_INVZWACXHZ6,NDAR_INVZWACXHZ6,09/09/2020,134,M,2_year_follow_up_y_arm_1,NaN,...,1,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,1.000000,1.5,1.2,3.0,prepubertal
39757,2573,112138,47118,NDAR_INVZWE6CD6Y,NDAR_INVZWE6CD6Y,08/08/2019,144,M,2_year_follow_up_y_arm_1,NaN,...,0,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,2.000000,2.5,2.2,6.0,midpubertal


In [8]:
puberty_c_df = calc_pubertal_scores(timepoint= '2_year_follow_up_y_arm_1', care_or_youth="caregiver")

In [9]:
puberty_c_df

,collection_id,abcd_ppdms01_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,sex,eventname,pds_select_language___1,...,menstrualcycle3_p,menstrualcycle4_p,menstrualcycle5_p,menstrualcycle6_p,collection_title,gonadal,adrenal,PDS_mean,PDS_cat_score,PDS_category
7,2573,55565,47106,NDAR_INV03BDCNWM,NDAR_INV03BDCNWM,06/25/2019,141,F,2_year_follow_up_y_arm_1,0,...,NaN,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,2.333333,2.5,2.4,6.0,midpubertal
18,2573,55576,47106,NDAR_INV03NW0RKL,NDAR_INV03NW0RKL,07/18/2019,146,F,2_year_follow_up_y_arm_1,0,...,NaN,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,2.333333,3.0,2.6,6.0,midpubertal
22,2573,55580,47106,NDAR_INV03XVEBPM,NDAR_INV03XVEBPM,08/12/2019,146,F,2_year_follow_up_y_arm_1,0,...,NaN,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,2.000000,1.0,1.6,3.0,early pubertal
26,2573,55584,47106,NDAR_INV03YU08FA,NDAR_INV03YU08FA,10/20/2020,137,F,2_year_follow_up_y_arm_1,0,...,NaN,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,2.333333,2.5,2.4,6.0,midpubertal
33,2573,55591,47106,NDAR_INV042UJKFB,NDAR_INV042UJKFB,12/10/2018,150,F,2_year_follow_up_y_arm_1,0,...,NaN,0.0,3.0,3.0,Adolescent Brain Cognitive Development Study (...,3.333333,3.0,3.2,6.0,late pubertal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39728,2573,95286,47106,NDAR_INVV2G5F6T0,NDAR_INVV2G5F6T0,02/21/2020,139,M,2_year_follow_up_y_arm_1,0,...,NaN,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,2.000000,2.0,2.0,6.0,midpubertal
39734,2573,95292,47106,NDAR_INVV2HGDTGK,NDAR_INVV2HGDTGK,01/31/2020,141,M,2_year_follow_up_y_arm_1,0,...,NaN,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,1.333333,2.0,1.6,3.0,prepubertal
39748,2573,95306,47106,NDAR_INVV2WL8NJZ,NDAR_INVV2WL8NJZ,08/21/2019,143,M,2_year_follow_up_y_arm_1,0,...,NaN,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,1.666667,1.5,1.6,4.0,early pubertal
39756,2573,95314,47106,NDAR_INVV2ZZ7DMV,NDAR_INVV2ZZ7DMV,02/02/2019,144,M,2_year_follow_up_y_arm_1,0,...,NaN,NaN,NaN,NaN,Adolescent Brain Cognitive Development Study (...,1.666667,1.0,1.4,3.0,prepubertal


In [36]:
puberty_y_df.to_csv('processedData\\PubertyCats.csv', index = False)

In [10]:
puberty_c_df.to_csv('processedData\\PubertyCatsCaregiver.csv', index = False)

# BMI, SES und race Covariates

In [72]:
_ , anthro2year = abcd_dataloader(file="abcd_ant01")

C:\Users\ninag\AppData\Local\Temp\ipykernel_13716\487761964.py:15: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../ABCDTabular/{}.txt".format(file),


In [73]:
anthro2year["avg_height12"] = anthro2year[["anthro_1_height_in","anthro2heightin"]].mean(axis=1, skipna=True)
anthro2year["avg_weight12"] = anthro2year[["anthroweight1lb","anthroweight2lb"]].mean(axis=1, skipna=True)
anthro2year["bmi"] = anthro2year["avg_weight12"] / (anthro2year["avg_height12"]**2) * 703

In [74]:
## exclude 10 < bmi < 50  
anthro2year = anthro2year[anthro2year["bmi"].between(10,50, inclusive="both")]

In [59]:
ses_base, _ = abcd_dataloader(file="pdem02") # there are no 2year follow up data

cols = ["demo_prnt_ed_v2", "demo_comb_income_v2"]


ses_base[cols].replace([777, 999], np.nan, inplace=True) # replace refuse and dont know with nan

for col in cols:
    temp_array = np.array(ses_base[col]).reshape(-1,1) # palm_inormal requires 2D array
    ses_base[col] = palm_inormal(temp_array)
ses_base["ses"] = ses_base[cols].mean(axis=1, skipna=True) # calculate mean to generate a single SES variable

In [70]:
race_ethnic, _ = abcd_dataloader(file="acspsw03") ### race / ethnicity 
# 1 = White; 2 = Black; 3 = Hispanic; 4 = Asian; 5 = Other 

In [75]:
covars_fo = anthro2year[["subjectkey", "bmi"]].merge(race_ethnic[["subjectkey", "race_ethnicity"]], on="subjectkey").merge(
    ses_base[["subjectkey","ses"]], on="subjectkey")
print(covars_fo.shape)

(7541, 4)


In [82]:
def switch_case(r):
    if r == '':
        return None
    elif r == 1:
        return 'white'
    elif r == 2:
        return 'black'
    elif r == 3:
        return 'hispanic'
    elif r == 4:
        return 'asian'
    elif r == 5:
        return 'other'
    else:
        return None
    

In [83]:
# Apply the function to create 'Status' column
covars_fo['ethno'] = covars_fo['race_ethnicity'].apply(switch_case)

covars_fo.head()

,subjectkey,bmi,race_ethnicity,ses,ethno
0,NDAR_INV02RCED7D,22.027809,1.0,-0.555621,white
1,NDAR_INV02UVMTY7,23.362143,1.0,0.783060,white
2,NDAR_INV02WP3TP6,31.830188,1.0,-0.622633,white
3,NDAR_INV030W95VP,26.037037,3.0,-1.377108,hispanic
4,NDAR_INV0387TP8R,18.030030,1.0,0.872865,white


In [88]:
covars_fo.to_csv('processedData\\SESBMIrace.csv', index = False)